In [ ]:
#load the data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#Define the softmax activation function
def softmax(z):
    z_exp = np.exp(z)
    z_sum = np.sum(z_exp, axis=0,keepdims=True)
    return z_exp / z_sum

In [ ]:
#Define the sigmoid activation function
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))   

In [ ]:
#Define the sigmoidGradient function
def sigmoidGradient(A):
    return A * (1-A)

In [ ]:
#Define the tanh activation function
def tanh(z):
    return (np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z))

In [ ]:
#Define the tanh gradient function
def tanhGradient(A):
    return 1 - np.square(A)

In [ ]:
#Define the ReLu activation function
def relu(z):
    return np.maximum(0,z)

In [ ]:
#Define Relu gradient function
def reluGradient(A):
    A[A < 0] = 0
    A[A >= 0] = 1
    return A

In [ ]:
#Define Leaky Relu function
def leakyrelu(z):
    return np.maximum(0.01*z , z)

In [ ]:
#Define LeakyRelu Gradient function
def leakyreluGradient(A):
    A[A < 0] = 0.01
    A[A >= 0] = 1
    return A

In [ ]:
def forwardPropagation(X,activations,parameters,keep_prob):
    
    cache={}
    cache["A0"]=X
    n=len(parameters)//2
    
    if keep_prob[0] < 1:
            Ashape = cache["A0"].shape
            cache["D0"] = np.random.rand(Ashape[0],Ashape[1])
            cache["D0"] = (cache["D0"] < keep_prob[0]).astype(int).astype(int)
            cache["A0"] = cache["A0"] * cache["D0"]
            cache["A0"] = cache["A0"] / keep_prob[0]          
    
    for i in range(1,n+1):
        cache["Z"+str(i)] = np.dot(parameters["W"+str(i)],cache["A"+str(i-1)]) + parameters["b"+str(i)]
        
        if activations[i-1]=="relu":
            cache["A"+str(i)] = relu(cache["Z"+str(i)])
        elif activations[i-1]=="tanh":
            cache["A"+str(i)] = tanh(cache["Z"+str(i)])
        elif activations[i-1]=="sigmoid":
            cache["A"+str(i)] = sigmoid(cache["Z"+str(i)])
        elif activations[i-1]=="leakyrelu":
            cache["A"+str(i)] = leakyrelu(cache["Z"+str(i)])
        elif activations[i-1]=="softmax":
            cache["A"+str(i)] = softmax(cache["Z"+str(i)])
            
        # For dropout regularization
        if keep_prob[i] < 1:
            Ashape = cache["A"+str(i)].shape
            cache["D"+str(i)] = np.random.rand(Ashape[0],Ashape[1])
            cache["D"+str(i)] = (cache["D"+str(i)] < keep_prob[i]).astype(int)
            cache["A"+str(i)] = cache["A"+str(i)] * cache["D"+str(i)]
            cache["A"+str(i)] = cache["A"+str(i)] / keep_prob[i]                 
    
            
    Yhat = cache["A"+str(i)]
    return cache ,Yhat

In [ ]:
#Compute cost without Regularization
def computeCost(Yhat,Y,lambd,parameters,softmax):
    m=Y.shape[1]
    logLoss = -(Y * np.log(Yhat))
    if not softmax:
        logLoss += -((1-Y) * np.log(1-Yhat))
    cost = (np.sum(logLoss))/m
    
    if lambd !=0:
        n = len(parameters)//2
        L2_regularization_cost=0
        for i in range(1,n+1):
            L2_regularization_cost +=  np.sum(np.square(parameters["W"+str(i)]))
            
        L2_regularization_cost *= lambd / (2 * m)
        cost += L2_regularization_cost
    
    return cost    

In [ ]:
def backwardPropagation(parameters,cache,X,Y,lambd,keep_prob):
    m=Y.shape[1]
    n = len(parameters)//2 
    Yhat = cache["A"+str(n)]
    if activations[n-1]=="softmax":
        dYhat = -Y/Yhat
    else: #sigmoid
        dYhat = -Y/Yhat + (1-Y)/(1-Yhat)
    grads={}
    
    grads["dA" + str(n)] = dYhat
    if keep_prob[n] < 1:
        grads["dA" + str(n)] = (grads["dA" + str(n)] * cache["D" + str(n)])/ keep_prob[n]
    
    for i in range(n,0,-1): 
        
        activationGradient = np.zeros(cache["A"+str(i)].shape)
        
        if activations[i-1]=="sigmoid":
            activationGradient = sigmoidGradient(cache["A"+str(i)])
        elif activations[i-1]=="relu":
            activationGradient = reluGradient(cache["A"+str(i)])
        elif activations[i-1]=="leakyrelu":
            activationGradient = leakyreluGradient(cache["A"+str(i)])
        elif activations[i-1]=="tanh":
            activationGradient = tanhGradient(cache["A"+str(i)])       
            
        
        if activations[i-1]=="softmax":
            grads["dZ" + str(i)] = cache["A"+str(i)] - Y            
        else:
            grads["dZ" + str(i)] = activationGradient * grads["dA" + str(i)]
            
        grads["dW" + str(i)] = np.dot(grads["dZ" + str(i)] , np.transpose(cache["A"+str(i-1)]))/m
        grads["db" + str(i)] = np.sum(grads["dZ" + str(i)] , axis=1,keepdims=True)/m
        grads["dA" + str(i-1)] = np.dot(np.transpose(parameters["W"+str(i)]) ,grads["dZ" + str(i)]) 
        
        if keep_prob[i-1] < 1:
            grads["dA" + str(i-1)] = (grads["dA" + str(i-1)] * cache["D" + str(i-1)])/keep_prob[i-1]
        
        if lambd != 0:
            grads["dW" + str(i)] = grads["dW" + str(i)] + (lambd/m * parameters["W" + str(i)])
            
        
    return grads

In [ ]:
def updateParametersWithAdam(parameters, grads, learning_rate,beta1,beta2,t,epsilon,V,S):
    
    n = len(parameters)//2 #number of layers
    for i in range(1,n+1):
               
        V["Vdw"+str(i)] = beta1 * V["Vdw"+str(i)] + (1-beta1) * grads["dW" + str(i)]
        V["Vdb"+str(i)] = beta1 * V["Vdb"+str(i)] + (1-beta1) * grads["db" + str(i)]
        
        S["Sdw"+str(i)] = beta2 * S["Sdw"+str(i)] + (1-beta2) * grads["dW" + str(i)] * grads["dW" + str(i)]
        S["Sdb"+str(i)] = beta2 * S["Sdb"+str(i)] + (1-beta2) * grads["db" + str(i)] * grads["db" + str(i)]
        
        VdwCorrected = V["Vdw"+str(i)]/(1- np.power(beta1,t))
        SdwCorrected = S["Sdw"+str(i)]/(1- np.power(beta2,t))
        VdbCorrected = V["Vdb"+str(i)]/(1- np.power(beta1,t))
        SdbCorrected = S["Sdb"+str(i)]/(1- np.power(beta2,t))        
        
        parameters["W"+str(i)] -= learning_rate * VdwCorrected / np.sqrt(SdwCorrected +epsilon)
        parameters["b"+str(i)] -= learning_rate * VdbCorrected / np.sqrt(SdbCorrected +epsilon)
   
    return parameters,V,S 

In [ ]:
def initializeAdam(parameters):    
    n = len(parameters)//2 #number of layers
    V={}
    S={}    
    for i in range(1,n+1):               
        V["Vdw"+str(i)] = np.zeros(parameters["W" + str(i)].shape)
        V["Vdb"+str(i)] = np.zeros(parameters["b" + str(i)].shape) 
        S["Sdw"+str(i)] = np.zeros(parameters["W" + str(i)].shape) 
        S["Sdb"+str(i)] = np.zeros(parameters["b" + str(i)].shape) 
    return V,S   

In [ ]:
def intitializeMomentum(parameters):    
    V={}    
    n = len(parameters)//2 #number of layers
    for i in range(1,n+1):               
        V["Vdw"+str(i)] = np.zeros(parameters["W" + str(i)].shape)
        V["Vdb"+str(i)] = np.zeros(parameters["b" + str(i)].shape)        
    return V    

In [ ]:
def updateParametersWithMomentum(parameters, grads, learning_rate,beta1,V):
    n = len(parameters)//2 #number of layers
    for i in range(1,n+1):
               
        V["Vdw"+str(i)] = beta1 * V["Vdw"+str(i)] + (1-beta1) * grads["dW" + str(i)]
        V["Vdb"+str(i)] = beta1 * V["Vdb"+str(i)] + (1-beta1) * grads["db" + str(i)]
        
        parameters["W"+str(i)] -=  learning_rate * V["Vdw"+str(i)]
        parameters["b"+str(i)] -=  learning_rate * V["Vdb"+str(i)]
   
    return parameters,V   

In [ ]:
def updateParameters(parameters, grads, learning_rate):
    n = len(parameters)//2 #number of layers
    for i in range(1,n+1):
        parameters["W"+str(i)] = parameters["W"+str(i)] - learning_rate * grads["dW" + str(i)]
        parameters["b"+str(i)] = parameters["b"+str(i)] - learning_rate * grads["db" + str(i)]
   
    return parameters    

In [ ]:
def initializeParameters(layers,activation):
    parameters={}
    for i in range(1,len(layers)):
       
        parameters["b"+str(i)] = np.zeros((layers[i],1))
        
        if activation[i-1]=="relu":
            parameters["W"+str(i)] = np.random.randn(layers[i],layers[i-1]) * np.sqrt(2/layers[i-1]) # He Initialization     
        else:
            parameters["W"+str(i)] = np.random.randn(layers[i],layers[i-1]) * np.sqrt(1/layers[i-1]) # Xavier Initialization
            
    return parameters

In [ ]:
#Choose an option for learning rate decay
def learningRateDecay(epoch_num,option):
    
    alpha0=1.2 #Tunable hyperparameters
    k=0.95 #Tunable hyperparameters
    
    l=0
    if option == 1:
        l = alpha0 / (1 + k * epoch_num)
    elif option == 2:    
        l = np.power(k,epoch_num) * alpha0  #Exponential Decay
    elif option == 3:    
        l = k * alpha0 / np.sqrt(epoch_num)
    else:     
        # Staircase
        t = epoch_num // 100
        if t <= 4: l = 1.2
        else: l = 0.001
    
    return l   

In [ ]:
def randomMiniBatches(X,Y,mini_batch_size):
    
    mini_batches = []
    m = X.shape[1]
    num_complete_minibatches = m//mini_batch_size
    
    # Shuffle (X, Y) synchronously    
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation]
    
    # Partition (shuffled_X, shuffled_Y). Minus the end case.       
    for k in range(0, num_complete_minibatches):
        mini_batch_X = X[:,k * mini_batch_size : (k+1) * mini_batch_size ]
        mini_batch_Y = Y[:,k * mini_batch_size : (k+1) * mini_batch_size ]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
        
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = X[:,(k+1) * mini_batch_size:]
        mini_batch_Y = Y[:,(k+1) * mini_batch_size:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def model(Xtrain,Ytrain,x_test,y_test,activations,layers,lambd,keep_prob,
          haveLearningRateDecay=False,DecayOption=0,optimizer="adam",mini_batch_size=64,beta1=0.9,beta2=0.999,
          epsilon=1e-8,learning_rate=0.001,max_epochs=60000,print_cost=False,print_accuracy=True): 
    
    #Initialize the parameters
    parameters = initializeParameters(layers,activations)
    if optimizer =="momentum":
        V = intitializeMomentum(parameters)
    if optimizer == "adam":
        V,S = initializeAdam(parameters)
    
    # initializing the counter required for Adam update
    t = 0       
            
    keep_probTest = [1] * len(keep_prob)
    improvementpossible = True
    i = 0 #Epoch Counter
    while(improvementpossible and i < max_epochs):        
        
        minibatches = randomMiniBatches(Xtrain.to_numpy(), Ytrain, mini_batch_size)
        cost_sum =0
        
        i += 1
        for minibatch in minibatches:
            
            #Get the mini batches
            (X, Y) = minibatch 
            
            #Forward Propagation
            cache ,Yhat = forwardPropagation(X,activations,parameters,keep_prob)
            
            #Compute Cost
            if print_cost:
                cost_sum += computeCost(Yhat,Y,lambd,parameters,activations[-1]=="softmax")
                    
            #Back Propagation
            grads = backwardPropagation(parameters,cache,X,Y,lambd,keep_prob)
            
            #Get the learning rate
            if haveLearningRateDecay:
                learning_rate = learningRateDecay(i,DecayOption)
            
            #Update Parameters
            if optimizer == "gd":
                parameters = updateParameters(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters,V = updateParametersWithMomentum(parameters, grads, learning_rate,beta1,V)
            elif optimizer =="adam":
                t += 1
                parameters,V,S = updateParametersWithAdam(parameters, grads, learning_rate,beta1,beta2,t,epsilon,V,S)
            
        # Print the cost every 1000 epochs
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_sum/len(minibatches))) 
            
        #Print accuracy 
        if print_accuracy and i % 100 == 0:    
            train_acc,test_acc = printTrainTestAccuracy(Xtrain,Ytrain,x_test,y_test,activations,parameters,keep_probTest,i)
            #if test_acc >0.99:
                #improvementpossible = False
                
    if not print_accuracy:           
        _,_ = printTrainTestAccuracy(Xtrain,Ytrain,x_test,y_test,activations,parameters,keep_probTest,i)        
        

    return parameters 

In [ ]:
def printTrainTestAccuracy(Xtrain,Ytrain,x_test,y_test,activations,parameters,keep_probTest,i):
    
    train_acc = predictAndEvaluate(Xtrain,Ytrain,activations,parameters,keep_probTest)
    test_acc = predictAndEvaluate(x_test,y_test,activations,parameters,keep_probTest)
    print("Train/test accuracy after epoch %i: %f : %f" %(i,train_acc,test_acc))
    
    return train_acc,test_acc

In [ ]:
def predictAndEvaluate(X,Y,activations,parameters,keep_probTest):
    #Predict 
    _ ,Yhat =forwardPropagation(X,activations,parameters,keep_probTest)

    #evaluation 
    m = X.shape[1]
    yhat = np.argmax(Yhat, axis=0)
    y = np.argmax(Y, axis=0)
    correct = np.sum(yhat == y) 
    return correct/m

In [ ]:
#Normalizing the inputs
def normalizeInputs(x_train,epsilon=1e-8):
    mean = np.sum(x_train.to_numpy(), axis=1,keepdims=True)/x_train.shape[1] # calculating the mean for every feature
    std = np.sqrt(np.sum(np.power(x_train.to_numpy(),2), axis=1,keepdims=True))/x_train.shape[1] # calculating the standard deviation for every feature
    x_train = x_train - mean / (std +epsilon)
    return x_train,mean,std,epsilon

In [ ]:
#Prepare the datasets for train and test

csv_path = "C:/Users/anubha/Downloads/digit-recognizer/train.csv"
df=pd.read_csv(csv_path)

#Create Y with 0s and 1s corresponding to a particular label
#So Y will be of size K * m
K=df["label"].value_counts().count()#number of distinct labels
m=df.shape[0] # no.of training points
    
Y = np.zeros((m,K))
for i in range(0,m):
    j = df.loc[i,"label"]
    Y[i,j] = 1
    
#Input
X=df.drop('label', 1)

#Train test split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)
x_train = np.transpose(x_train)
x_test = np.transpose(x_test)
y_train = np.transpose(y_train)
y_test = np.transpose(y_test)

    
# Normalize the input data
#x_train,mean,std,epsilon = normalizeInputs(x_train)
#x_test = (x_test - mean) / (std + epsilon)      

In [ ]:
# select the number of units and the activation function of each unit
layers = [X.shape[1],50,50,50,Y.shape[1]] # in X, Y data is not transposed, so columns have features.
activations = ["sigmoid","sigmoid","sigmoid","sigmoid"] 
keep_prob = [1,1,1,1,1]

parameters={}
for lambd in [0.01]:
    print("lambd = ",lambd)
    #Get the parameters for the model
    parameters = model(x_train,y_train,x_test,y_test,activations,layers,lambd,keep_prob)
    

In [ ]:
#Prediction and submission
test_csv_path="C:/Users/anubha/Downloads/digit-recognizer/test.csv"
dftest=pd.read_csv(test_csv_path)
X_predict=np.transpose(dftest)
#X_predict = (X_predict - mean) / (std + epsilon)
keep_probTest = [1] * len(keep_prob)
_ ,Yhat =forwardPropagation(X_predict,activations,parameters,keep_probTest)
dftest["Label"]=np.argmax(Yhat, axis=0)

df_submit=dftest["Label"]
df_submit.to_csv("C:/Users/anubha/Desktop/digitRecognition.csv")